In [8]:
import numpy as np
import pandas as pd
import torch
import torchvision
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

In [3]:
# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

In [4]:
batch_size = 256
# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [5]:
# Define the CNN
class Fashion(torch.nn.Module):
    
    def __init__(self):
        super(Fashion,self).__init__()
        
        # define the convolution layers
        self.conv1 = torch.nn.Conv2d(in_channels = 1, out_channels = 32, kernel_size = 7)
        self.pool1 = torch.nn.MaxPool2d(kernel_size=2)
        self.drop1 = torch.nn.Dropout2d(p=0.25)
        
        self.conv2 = torch.nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3)
        self.pool2 = torch.nn.MaxPool2d(kernel_size=2)
        self.drop2 = torch.nn.Dropout2d(p=0.25)

        self.conv3 = torch.nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3)
        self.drop3 = torch.nn.Dropout2d(p=0.4)

        self.fc4 = torch.nn.Linear(2*2*128, 128)
        self.drop4 = torch.nn.Dropout2d(p=0.3)
        
        self.fc5 = torch.nn.Linear(128, 10)
    
    def forward(self,x):
        
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = self.drop1(x)
        
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = self.drop2(x)
        
        x = F.relu(self.conv3(x))
        x = self.drop3(x)
        
        #Recall that the -1 infers this dimension from the other given dimension
        x = x.view(-1, 2*2*128)
        
        x = F.relu(self.fc4(x))
        x = self.drop4(x)

        #(activation applied later)
        x = self.fc5(x)
        
        return x        

In [6]:
#instance of the Conv Net
cnn = Fashion()
#loss function and optimizer for updating weights
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.001)

In [9]:
# Train the CNN
losses = [];
num_epochs = 4
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.float())
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        losses.append(loss.data[0]);
        
        if (i+1) % 100 == 0:
            print ('Epoch : %d/%d, Iter : %d/%d,  Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

/Users/ajaysingh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  app.launch_new_instance()
/Users/ajaysingh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch : 1/4, Iter : 100/234,  Loss: 0.2676
Epoch : 1/4, Iter : 200/234,  Loss: 0.2206
Epoch : 2/4, Iter : 100/234,  Loss: 0.1538
Epoch : 2/4, Iter : 200/234,  Loss: 0.1277
Epoch : 3/4, Iter : 100/234,  Loss: 0.0998
Epoch : 3/4, Iter : 200/234,  Loss: 0.1118
Epoch : 4/4, Iter : 100/234,  Loss: 0.0388
Epoch : 4/4, Iter : 200/234,  Loss: 0.0741


In [10]:
# Evaluate on Test Set
pred=[]
true_label=[]
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = cnn(images)
        _, predicted = torch.max(outputs.data, 1)
        
        # convert tensors to numpy arrays
        pred.append(predicted.numpy())
        true_label.append(labels.numpy())
        
pred = np.concatenate(pred, axis=0 )
true_label = np.concatenate(true_label, axis=0)

print(classification_report(true_label, pred))

             precision    recall  f1-score   support

          0       0.98      0.99      0.98       980
          1       0.99      0.98      0.99      1135
          2       0.98      0.97      0.98      1032
          3       0.97      0.98      0.97      1010
          4       0.98      0.98      0.98       982
          5       0.98      0.97      0.97       892
          6       0.99      0.97      0.98       958
          7       0.96      0.97      0.97      1028
          8       0.95      0.97      0.96       974
          9       0.96      0.97      0.97      1009

avg / total       0.98      0.98      0.98     10000

